# Analyse de données de films

Nous proposons de travailler sur des données décrivant des films. Les possibilités sont larges et vous êtes évalués sur vos propositions et votre méthodologie plus que sur vos résultats.

Les données de départ sont disponibles sur:
https://grouplens.org/datasets/movielens/
au format CSV. 

Nous nous intéresserons en particulier au jeu de données: **MovieLens 20M Dataset**. Dans ce jeu de données, vous disposez entre autre de:
* Idendifiant du film dans IMdb et TMdb (ça sera important ensuite)
* Catégorie(s) du film
* Titre du film
* Notes données par les internautes aux films

Afin de rendre le projet plus intéressant, nous ajoutons des données sur les acteurs et producteurs associés aux films (récupéré sur TMdb). Ces données sont disponibles sur les liens suivants:

http://webia.lip6.fr/~guigue/film_v2.pkl <br>
http://webia.lip6.fr/~guigue/act_v2.pkl <br>
http://webia.lip6.fr/~guigue/crew_v2.pkl

Ces fichiers contiennent respectivement : une nouvelle description des films (dont l'identifiant TMdb et la note moyenne donnée par les internautes, la date de sortie,...), une description des acteurs de chaque film et une description des équipes (scénariste, producteur, metteur en scène) pour chaque film.

Ces données sont des listes de taille 26908, chaque élément de la liste correspondant à un dictionnaire dont vous étudierez les clés pour récupérer les informations utiles.

**ATTENTION** Les contraintes de récupération d'informations en ligne font que la base MovieLens compte 27278 films mais les fichiers ci-dessus n'en comptent que 26908. Le plus simple est probablement d'éliminer les films de MovieLens qui ne sont pas dans cette seconde base.

## Consignes générales pour l'analyse des données

Vous devez proposer plusieurs analyses des données, qui devront à minima utiliser les
 techniques suivantes:
 
1. Mettre en forme les données pour identifier les acteurs et les catégories, les indexer
1. Traiter au moins un problème de régression supervisé (par exemple la prédiction de la note moyenne donnée à un film par les internautes).
1. Traiter au moins un problème de classification supervisé (par exemple la prédiction de la catégorie d'un film)
1. Utiliser les données catégorielles (catégories, acteurs,...) de manière discrète ET de manière coninue (*dummy coding*) dans des approches différentes
1. Proposer au moins une approche de catégorisation non supervisée (pour regrouper les acteurs par exemple)
1. Mener une campagne d'expérience permettant de comparer les performances sur un problème en fonction des valeurs d'un paramètre (et donc, in fine, trouver la meilleure valeur du paramètre)
1. Proposer quelques illustrations

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import pickle as pkl
import sys
import math
sys.path.append('../')
import iads as iads
from iads import LabeledSet as ls
from iads import Classifiers as cl
from iads import util_iads as ui
from iads import engineering as en

## Chargement des données (base MovieLens + enrichissements)

In [2]:
# Chargement des données MovieLens
links = pd.read_csv("data/ml-20m/links.csv", encoding='utf8')
movies = pd.read_csv("data/ml-20m/movies.csv", encoding='utf8')
ratings = pd.read_csv("data/ml-20m/ratings.csv", encoding='utf8')
tags = pd.read_csv("data/ml-20m/tags.csv", encoding='utf8')
genomeS = pd.read_csv("data/ml-20m/genome-scores.csv", encoding='utf8')
genomeT = pd.read_csv("data/ml-20m/genome-tags.csv", encoding='utf8')

In [3]:
# Chargement des données complémentaires
acteurs = pkl.load(open("data/act_v2.pkl", "rb"))
equipes = pkl.load(open("data/crew_v2.pkl", "rb"))
films = pkl.load(open("data/film_v2.pkl", "rb"))

<h2>Filtrage des données</h2>

In [4]:
"""
print("BEFORE")
print(len(movies))
#print(len(films))
moviesFilter = []
filmId = []
cpt = 0
for i in range(len(films)):
    filmId.append(films[i]["id"])
    
for i in range(len(movies)):
    line = links.loc[links['movieId'] == movies.iloc[i]["movieId"]]
    value = float(line.iloc[0]["tmdbId"])
    if(math.isnan(value)):
        cpt += 1
        continue
    if(value in filmId):
        moviesFilter.append(movies.iloc[i]["movieId"])
    else:
        cpt += 1
movies = movies[movies.movieId.isin(moviesFilter)]
links = links[links.movieId.isin(moviesFilter)]
ratings = ratings[ratings.movieId.isin(moviesFilter)]
print("\nAFTER")
print(len(movies))
movies.to_csv("data/ml-20m/movies.csv", encoding='utf8')
"""

'\nprint("BEFORE")\nprint(len(movies))\n#print(len(films))\nmoviesFilter = []\nfilmId = []\ncpt = 0\nfor i in range(len(films)):\n    filmId.append(films[i]["id"])\n    \nfor i in range(len(movies)):\n    line = links.loc[links[\'movieId\'] == movies.iloc[i]["movieId"]]\n    value = float(line.iloc[0]["tmdbId"])\n    if(math.isnan(value)):\n        cpt += 1\n        continue\n    if(value in filmId):\n        moviesFilter.append(movies.iloc[i]["movieId"])\n    else:\n        cpt += 1\nmovies = movies[movies.movieId.isin(moviesFilter)]\nlinks = links[links.movieId.isin(moviesFilter)]\nratings = ratings[ratings.movieId.isin(moviesFilter)]\nprint("\nAFTER")\nprint(len(movies))\nmovies.to_csv("data/ml-20m/movies.csv", encoding=\'utf8\')\n'

<h2>Visionnage des données</h2>

In [10]:
##links
#ratings
#tags
#genomeS
#genomeT
#movies
#films[9]
#acteurs[0]
#equipes[0][194]

## Feature engineering
A vous de créer les caractéristiques de description des données qui permettront d'améliorer les performances dans les tâches que vous aurez choisi d'aborder dans le projet.

In [6]:
# Faire un dictionnaire avec tous les acteurs (acteur => indice)
# + un dictionnaire inversé (indice => acteur)
ae = en.ActorsEngineering(acteurs)
ae.reverse()
print("Tom Hanks à joué dans", ae.playedMovies["Tom Hanks"], "films")

ActorsEngineering init in process
ActorsEngineering init successful
Tom Hanks à joué dans 57 films


In [7]:
#Faire un dataframe sur les categories, leur note moyenne et leur engagement
ge = en.GenresEngineering(movies,(links, films))
print("nbFilms:", ge.nbFilms, "\n")
print("moyenne note:", ge.averageRating, "\n")
print("engagements:", ge.ratingCount, "\n")

GenresEngineering init in process
GenresEngineering init successful
nbFilms: {'Adventure': 2264, 'Animation': 986, 'Children': 1125, 'Comedy': 8253, 'Fantasy': 1372, 'Romance': 4057, 'Drama': 13079, 'Action': 3463, 'Crime': 2892, 'Thriller': 4126, 'Horror': 2578, 'Mystery': 1476, 'Sci-Fi': 1692, 'IMAX': 195, 'Documentary': 2331, 'War': 1162, 'Musical': 1012, 'Western': 664, 'Film-Noir': 329, '(no genres listed)': 233} 

moyenne note: {'Adventure': 6.89248789221955, 'Animation': 7.072643748377346, 'Children': 6.823645059838005, 'Comedy': 6.634316435901211, 'Fantasy': 6.86253529590938, 'Romance': 6.884282613263715, 'Drama': 7.133865521572034, 'Action': 6.755422691063391, 'Crime': 7.078060614533, 'Thriller': 6.762631128193778, 'Horror': 6.362509240078424, 'Mystery': 7.096901917123273, 'Sci-Fi': 6.835237747138164, 'IMAX': 6.957103073718686, 'Documentary': 7.184183999432101, 'War': 7.285639627511125, 'Musical': 7.089185958316838, 'Western': 7.062982669358924, 'Film-Noir': 7.411919388842293,

In [8]:
df = ge.toDataFrame()
df

,name,quantite,note,engagement
0,Adventure,2264,6.892488,1826305
1,Animation,986,7.072644,654790
2,Children,1125,6.823645,726545
3,Comedy,8253,6.634316,2190175
4,Fantasy,1372,6.862535,971784
5,Romance,4057,6.884283,1051727
6,Drama,13079,7.133866,2792433
7,Action,3463,6.755423,2350097
8,Crime,2892,7.078061,1101617
9,Thriller,4126,6.762631,1933604
